# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [19]:
# импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

In [20]:
# загружу все датасеты, выведу первые 5 строк и информацию
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
display(data_0.head())
data_0.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [21]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
display(data_1.head())
data_1.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [22]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
display(data_2.head())
data_2.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [23]:
for i in (data_0,data_1,data_2):
    display(i.duplicated().sum())

0

0

0

**Вывод**

В каждом датасете содержится информация по 100 тысячам скважинам. Данные без пропусков, в формате десятичных чисел. Общих дубликатов нет.

## Обучение и проверка модели

In [24]:
# для удобства прогнозирования напишу общую функцию
def predictions_region(df):
    X = df.drop(['id','product'],axis = 1) 
    target = df['product']
    
    X_train, X_valid, target_train, target_valid = train_test_split(X, target, test_size = 0.25, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    model = LinearRegression().fit(X_train_scaled, target_train)
    predictions = model.predict(X_valid_scaled)
    
    rmse = (mean_squared_error(target_valid, predictions, squared=False))

    stock_mean = df['product'].mean()
    stock_mean_pred = predictions.mean()
    
    target_valid = target_valid.reset_index(drop=True)
    
    return predictions, rmse, stock_mean, stock_mean_pred, target_valid

In [25]:
predictions_0, rmse_0, stock_mean_0, stock_mean_pred_0, target_valid_0 = predictions_region(data_0)

print('RMSE модели = {:.2f}'.format(rmse_0))
print('Средний запас сырья = {:.2f} тыс. баррелей'.format(stock_mean_0))
print('Средний предсказанный запас сырья {:.2f} тыс. баррелей'.format(stock_mean_pred_0))

RMSE модели = 37.76
Средний запас сырья = 92.50 тыс. баррелей
Средний предсказанный запас сырья 92.40 тыс. баррелей


In [26]:
predictions_1, rmse_1, stock_mean_1, stock_mean_pred_1, target_valid_1 = predictions_region(data_1)

print('RMSE модели = {:.2f}'.format(rmse_1))
print('Средний запас сырья = {:.2f} тыс. баррелей'.format(stock_mean_1))
print('Средний предсказанный запас сырья {:.2f} тыс. баррелей'.format(stock_mean_pred_1))

RMSE модели = 0.89
Средний запас сырья = 68.83 тыс. баррелей
Средний предсказанный запас сырья 68.71 тыс. баррелей


In [27]:
predictions_2, rmse_2, stock_mean_2, stock_mean_pred_2, target_valid_2 = predictions_region(data_2)

print('RMSE модели = {:.2f}'.format(rmse_2))
print('Средний запас сырья = {:.2f} тыс. баррелей'.format(stock_mean_2))
print('Средний предсказанный запас сырья {:.2f} тыс. баррелей'.format(stock_mean_pred_2))

RMSE модели = 40.15
Средний запас сырья = 95.00 тыс. баррелей
Средний предсказанный запас сырья 94.77 тыс. баррелей


**Выводы**

- Лучший результат по RMSE показала модель для региона 1
- При этом в регионе 1 в 1,5 раза меньше запасов, чем в регионах 0 и 2

## Подготовка к расчёту прибыли

In [28]:
BUDGET = 10_000_000_000 #общий бюджет
TOTAL_LOCATIONS = 500 #при разведке региона исследуют 500 точек
TOTAL_BEST_LOCATIONS = 200  #200 лучших скважин 
PROFIT_FROM_SINGLE_PRODUCT = 450_000   #Доход с каждой единицы продукта 450 000 рублей

In [29]:
sufficient_volume = BUDGET / TOTAL_BEST_LOCATIONS / PROFIT_FROM_SINGLE_PRODUCT
print('Точка безубыточности = {:.2f} тыс. баррелей'.format(sufficient_volume))

Точка безубыточности = 111.11 тыс. баррелей


In [30]:
sufficient_volume_from_single = BUDGET / PROFIT_FROM_SINGLE_PRODUCT
print('Точка безубыточности для одной скважины = {:.2f} тыс. баррелей'.format(sufficient_volume_from_single ))

Точка безубыточности для одной скважины = 22222.22 тыс. баррелей


**Выводы**

Для выхода на безубыточность в каждом регионе, при заданных параметрах, количество добываемого сырья должно быть равно 111 тысяч баррелей. Средние запасы в трех регионах намного меньше - 92, 68 и 95 тысяч баррелей. Соответственно для извлечения прибыли от разработки скважин следует проанализировать сами скважины - спрогнозировать возможную прибыль по каждой скважине и отобрать прибыльные, то есть разрабатывать не все точки.

## Расчёт прибыли

In [31]:
# сделаю функцию для расчета прибыли
def revenue(target, predictions, number):
    probs_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[probs_sorted.index][:number]
    return (PROFIT_FROM_SINGLE_PRODUCT * selected.sum() - BUDGET)

In [32]:
revenue_0 = revenue(target_valid_0,predictions_0,200)
revenue_1 = revenue(target_valid_1,predictions_1,200)
revenue_2 = revenue(target_valid_2,predictions_2,200)

print('Прибыль по выбранным скважинам региона 0 = {:.2f} млн. руб.'.format(revenue_0/1000000))
print('Прибыль по выбранным скважинам региона 1 = {:.2f} млн. руб.'.format(revenue_1/1000000))
print('Прибыль по выбранным скважинам региона 2 = {:.2f} млн. руб.'.format(revenue_2/1000000))

Прибыль по выбранным скважинам региона 0 = 3359.14 млн. руб.
Прибыль по выбранным скважинам региона 1 = 2415.09 млн. руб.
Прибыль по выбранным скважинам региона 2 = 2598.57 млн. руб.


## Риски и прибыль с каждого региона

In [33]:
# функция для расчета Bootstrap
state = np.random.RandomState(42)
def calc_bootstrap(target,probabilities, region):
    values = []
    for i in range(1000):
        target_subsample = target.sample(500, replace = True, random_state = state)
        probs_subsample = probabilities[target_subsample.index] 
        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    prob_loss = (values < 0).mean()
    interval = (values.quantile(0.025), values.quantile(0.975))
    mean = values.mean()
    print(region)
    print('Средняя прибыль:{:.3f} млн. рублей'.format(mean/10**6))
    print('Доверительный интервал среднего:{}'.format(interval))
    print('2.5%-квантиль:{:.3f} млн. рублей'.format(lower/10**6))
    print('Риск:{:.2%}'.format(prob_loss))
    print('\n')
    
calc_bootstrap(target_valid_0, pd.Series(predictions_0), 'Регион №0')
calc_bootstrap(target_valid_1, pd.Series(predictions_1), 'Регион №1')
calc_bootstrap(target_valid_2, pd.Series(predictions_2), 'Регион №2')

Регион №0
Средняя прибыль:427.848 млн. рублей
Доверительный интервал среднего:(-97249829.56859529, 954215192.7088149)
2.5%-квантиль:-97.250 млн. рублей
Риск:5.50%


Регион №1
Средняя прибыль:511.530 млн. рублей
Доверительный интервал среднего:(91700564.13644528, 921455668.3285091)
2.5%-квантиль:91.701 млн. рублей
Риск:0.60%


Регион №2
Средняя прибыль:408.546 млн. рублей
Доверительный интервал среднего:(-120624872.94271684, 960859440.7253835)
2.5%-квантиль:-120.625 млн. рублей
Риск:7.50%




**Общие выводы**
Для разработки следует выбрать регион 1. У него запасов меньше, чем в остальных. Однако по средним предсказанным показателям он лучше остальных:
- Средняя прибыль по технике Bootstrap составляет 511 млн рублей
- Риск убытков самый низкий - всего 0.6%